In [3]:
import sys
import os
from dotenv import load_dotenv

sys.path.append(os.path.abspath(".."))
load_dotenv("../.env")

from app.rag.repository.meili import LangChainMeiliRepository
from app.core.config import settings

repo = LangChainMeiliRepository()

import meilisearch
import os

client = meilisearch.Client(settings.MEILI_HTTP_ADDR, settings.MEILI_KEY)

index_name = "langchain-demo" 
index = client.index(index_name)

index.update_embedders({
    "default": {
        "source": "userProvided",
        "dimensions": 1536
    }
})

/home/ba2slk/projects/catchup-rag-server/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TaskInfo(task_uid=262, index_uid='langchain-demo', status='enqueued', type='settingsUpdate', enqueued_at=datetime.datetime(2025, 12, 28, 16, 56, 36, 880277))

In [20]:
from app.rag.dependencies import get_vector_repository

repo = get_vector_repository()

repo.initialize()

# 테스트 질문과 정답 문서에만 있는 단어 쌍
test_cases = [
    {"query": "배고파 뭐먹지? 식대 나오는 걸로 얼마짜리 라면까지 먹을 수 있지?", "target_keyword": "13,000원"},
    {"query": "바쁜 시간 대의 재택근무 규정?", "target_keyword": "메신저 응답"},
]

ratios = [0.0, 0.3, 0.5, 0.7, 1.0] # 1에 가까울수록 semantic search 비중이 커짐.

print("결과")
print("-" * 30)

for ratio in ratios:
    score = 0
    print(f"\n[Ratio: {ratio}]")
    
    for case in test_cases:
        docs = repo.retrieve(case["query"], 3, ratio)
        
        found = any(case["target_keyword"] in doc.page_content for doc in docs)
        result = "Hit" if found else "Miss"
        
        print(f"  Q: {case['query']} -> {result}")
        if found: score += 1
            
    print(f"  => 정확도: {score}/{len(test_cases)}")

결과
------------------------------

[Ratio: 0.0]
  Q: 배고파 뭐먹지? 식대 나오는 걸로 얼마짜리 라면까지 먹을 수 있지? -> Hit
  Q: 바쁜 시간 대의 재택근무 규정? -> Hit
  => 정확도: 2/2

[Ratio: 0.3]
  Q: 배고파 뭐먹지? 식대 나오는 걸로 얼마짜리 라면까지 먹을 수 있지? -> Hit
  Q: 바쁜 시간 대의 재택근무 규정? -> Hit
  => 정확도: 2/2

[Ratio: 0.5]
  Q: 배고파 뭐먹지? 식대 나오는 걸로 얼마짜리 라면까지 먹을 수 있지? -> Hit
  Q: 바쁜 시간 대의 재택근무 규정? -> Hit
  => 정확도: 2/2

[Ratio: 0.7]
  Q: 배고파 뭐먹지? 식대 나오는 걸로 얼마짜리 라면까지 먹을 수 있지? -> Hit
  Q: 바쁜 시간 대의 재택근무 규정? -> Hit
  => 정확도: 2/2

[Ratio: 1.0]
  Q: 배고파 뭐먹지? 식대 나오는 걸로 얼마짜리 라면까지 먹을 수 있지? -> Hit
  Q: 바쁜 시간 대의 재택근무 규정? -> Hit
  => 정확도: 2/2
